In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

im = cv2.imread('out.jpg')

imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,127,255,cv2.THRESH_BINARY)
image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

cnt = contours[0]
x,y,w,h = cv2.boundingRect(cnt)

cv2.rectangle(im,(x-50,y-50),(x+w+50,y+h+50),(0,255,0),1)

plt.imshow(im, cmap='gray')
plt.show()


In [17]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX

while 1:
    ret, img = cap.read()
    
    imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
    
    ret,thresh = cv2.threshold(imgray,240,255,0)
    
    
    
    st1 = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 21), (10, 10))
    st2 = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11), (5, 5))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, st1) 
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, st2) 
    thresh = cv2.GaussianBlur(thresh, (5, 5), 2)
    

    image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    
    thr = 0
    #cnt = contours
    #cv2.drawContours(img,cnt,-1,(0,250,0),2)
    
    crop_img = None
    if len(contours) > 0:
        cnt = contours[0]
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.rectangle(img,(x-25,y-25),(x+w+25,y+h+25),(0,0,255),1)
        
        
        crop_img = img[y: y + h, x: x + w] # Crop from x, y, w, h -> 100, 200, 300, 400
        # NOTE: its img[y: y + h, x: x + w] and *not* img[x: x + w, y: y + h]
        
        
        cv2.putText(img,'3',(x,y-30), font, 1,(0,255,0),1,cv2.LINE_AA)
        
        
        imgray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        
        ret,thresh2 = cv2.threshold(imgray,127,255,cv2.THRESH_BINARY_INV)
        
        cv2.imshow("cropped", thresh2)

        
    
    
    cv2.imshow('img', img)
    cv2.imshow('img1', thresh)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [14]:
import cv2

# load the video
#camera = cv2.VideoCapture(args["video"])
camera = cv2.VideoCapture(0) # microscope camera
coeff=1.95 # microscope camera focus distance coeffitienf
Sr=10 # frame dimension to calculate distance

# keep looping
while True:
    # grab the current frame and initialize the status text
    (grabbed, frame) = camera.read()
    status = "No Targets"

    # convert the frame to grayscale, blur it, and detect edges
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    edged = cv2.Canny(blurred, 50, 150)
        
    cv2.imshow("edged", edged)
    
    
    
    
    
    
    # find contours in the edge map
    (_i, cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.01 * peri, True)

        # ensure that the approximated contour is "roughly" rectangular
        if len(approx) >= 4 and len(approx) <= 6:
            
            
            # compute the bounding box of the approximated contour and
            # use the bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            aspectRatio = w / float(h)

            # calculate distance to frame
            k=float(w)/640
            S = Sr*coeff/k

            # compute the solidity of the original contour
            area = cv2.contourArea(c)
            hullArea = cv2.contourArea(cv2.convexHull(c))
            solidity = area / float(hullArea)

            # compute whether or not the width and height, solidity, and
            # aspect ratio of the contour falls within appropriate bounds
            keepDims = w > 25 and h > 25
            keepSolidity = solidity > 0.3
            keepAspectRatio = aspectRatio >= 0.8 and aspectRatio <= 1.2

            # ensure that the contour passes all our tests
            if keepDims and keepSolidity and keepAspectRatio:
                # draw an outline around the target and update the status
                # text
                cv2.rectangle(frame,(x-25,y-25),(x+w+25,y+h+25),(0,0,255),1)
                #cv2.drawContours(frame, [approx], -1, (0, 0, 255), 4)
                status = "Target(s) Acquired"





    # draw the status text on the frame
    cv2.putText(frame, status, (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # show the frame and record if a key is pressed
    cv2.imshow("Frame", frame)
    
    
    key = cv2.waitKey(30) & 0xff
    if key == 27:
        break

# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()
